In [1]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')

# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

In [2]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,id,eventTimestamp,unitDisplayType,brandName,bundleId,countryCode,deviceId,osAndVersion,bidFloorPrice,sentPrice
0,1,1657758857892,rewarded,Generic,com.loop.match3d,US,6a0a94554cf,Android-4.0,0.05,0.06
1,2,1657210707978,interstitial,Generic,com.loop.match3d,ZA,6a0b0e59f45,Android-4.0,0.01,0.16
2,3,1657392939412,interstitial,Motorola,com.YayySAL.DodgeAgent,BR,6a0fa820c46,Android-10.0,0.05,0.06
3,4,1657386816882,interstitial,Huawei,com.tintash.nailsalon,IQ,6a142bdbea2,Android-9.0,0.01,0.05
4,5,1657211600823,banner,Generic,com.tilegarden.match3,US,6a16943a771,Android-4.0,0.01,0.03
...,...,...,...,...,...,...,...,...,...,...
41579,41580,1657731605063,banner,Generic,com.tilegarden.match3,NZ,80edf5837e8,Android-4.0,0.01,0.02
41580,41581,1657657131101,interstitial,Apple,1436213906,VN,80ee6a5eebf,iOS-13.6,0.01,0.90
41581,41582,1657156070854,rewarded,Generic,com.loop.match3d,US,80efe531afc,Android-4.0,0.05,0.69
41582,41583,1657659603568,banner,Apple,1502447854,US,80f02dd397f,iOS-15.0,0.01,0.10


## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [8]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        df = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~df["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

df


,id,bundleId,content,embedding
0,1,com.loop.match3d,"Get ready for a new, challenging and original ...","[0.31683534,0.625009,-0.120355725,0.21742287,-..."
1,2,com.YayySAL.DodgeAgent,"Your mission, should you choose to accept it, ...","[0.54112804,0.74467534,-0.5817448,0.95967793,0..."
2,3,com.tintash.nailsalon,It is manicure madness over here and it�s your...,"[0.31176662,0.6059465,-1.1395652,0.07631032,-0..."
3,4,com.tilegarden.match3,\nIf you enjoy playing Match 3 tile & mahjong ...,"[0.050924554,1.0967331,-0.3083164,0.13220677,-..."
4,6,com.AppIdeas.LevelUpRunner,"Are you a real hero? Are you the strongest, th...","[0.8975613,0.4644255,-0.18019181,1.2617059,0.0..."
5,17,1582745578,Dominate the coal industry and become the next...,"[0.8849048,0.65851974,-0.3386693,0.18323693,0...."
6,18,1569586264,Choose the appropriate outfit to make it throu...,"[0.20005329,0.36083457,-0.13792276,0.34308657,..."
7,5,com.kamilbilge.ropesavior3d,Become the hero we need in this tangled puzzle...,"[0.5112476,0.51280636,-0.05630458,1.2819328,0...."
8,7,se.ace.fishinc,Gather your riches and travel across the open ...,"[0.7309437,0.41121697,-0.61960155,0.5350136,0...."
9,8,com.volt.dresstoimpress,\nChoose the appropriate outfit to make it thr...,"[0.20005329,0.36083457,-0.13792276,0.34308657,..."


We can use the `<=>` operator to run vector search within the database

In [4]:

vec = json.loads(df.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

Embedding size: 1024


,bundleId
0,com.loop.match3d
1,1502447854
2,1529614832
3,com.tilegarden.match3
4,com.kamilbilge.ropesavior3d
5,1579489488
6,1436213906
7,1586795332
8,com.AppIdeas.LevelUpRunner
9,se.ace.fishinc


# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

In [5]:
# TODO: Please write your analysis here
import numpy as np

# Solution

We are going to check the hypothosis: apps with similar descriptions would have similar sentPrice.

For each app we compute the average sentPrice, and then for each pair of apps we compute the cosine similarity between the decription embeddings, and the absolute value of difference between the sentPrice averages.

We can do this using a single sql query. the result is a dataframe with the following columns:

- firstId: the bundleId of the first app
- secondId: the bundleId of the second app
- sentPriceDifference: | sentPrice average of first app - sentPrice average of second app |
- descripitonSimilarity: cosine similarity of embeddings of the two app's descriptions

Then we compute the pearson correlation of the sentPriceDifference and descripitonSimilarity. 
 
pearson correlation is a value between -1 and 1, where a value closer to 1 indicates a strong positive correlation, a value closer to -1 indicates a strong negative correlation, and a value closer to 0 indicates no correlation

In [2]:
sql_query = '''
    WITH apps AS (
        SELECT 
            auctions."bundleId",
            AVG(auctions."sentPrice") AS "avgSentPrice",
            app_vectors."embedding"
        FROM auctions
        LEFT JOIN app_vectors ON auctions."bundleId" = app_vectors."bundleId"
        GROUP BY auctions."bundleId", app_vectors."embedding"
    )
    
    SELECT 
        a1."bundleId" AS "firstId",
        a2."bundleId" AS "secondId",
        ABS(a1."avgSentPrice" - a2."avgSentPrice") AS "sentPriceDifference",
        a1.embedding <=> a2.embedding AS "descripitonSimilarity"

    FROM apps a1
    JOIN apps a2 ON a1."bundleId" < a2."bundleId"
    
;'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

,firstId,secondId,sentPriceDifference,descripitonSimilarity
0,1436213906,1502447854,2.449073,0.449619
1,1436213906,1523081624,3.616717,0.521262
2,1436213906,1529614832,1.596738,0.568431
3,1436213906,1542256628,2.603758,0.489775
4,1436213906,1569586264,14.181624,0.591438
...,...,...,...,...
148,com.volt.dresstoimpress,dovi.coalmining.inc,0.148056,0.622133
149,com.volt.dresstoimpress,se.ace.fishinc,1.386668,0.596807
150,com.YayySAL.DodgeAgent,dovi.coalmining.inc,0.147782,0.509314
151,com.YayySAL.DodgeAgent,se.ace.fishinc,1.386393,0.514294


In [9]:
correlation_matrix = np.corrcoef(df['sentPriceDifference'].values, df['descripitonSimilarity'].values)
pearson_correlation_coefficient = correlation_matrix[0, 1]
pearson_correlation_coefficient

0.0633181649323078

### The pearson correlation we got from the dataset is: 0.063

this value is closer to 0, which means that there is no correlation between the cosine similarity of the app's description and the difference between their sentPrice. Which means our hypothesis is false.

Note: We can formally show that our alternative hypothesis (pearson coefficient > 0) p-value is not small enough to reject the null hypothesis (pearson coefficient = 0)

# Importance of each feature

while we have shown that based on the dataset we have there is not enough evidence to back our hypothesis. And we have shown that there is no linear correlation between the embedding cosine similarity and sentPrice.

We are going to investigate the importance of the embedding feature further, by using decision tree to predict the value of sentPrice.

We will train the model with all the features at first and calculate the mse (mean squared error).

Then for every feature, we will drop it, retrain the model, and check how that impacts the mse.

### Preprocessing

In [24]:
# fetch the auctions from database

sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    auctions = pd.read_sql(sql_query, con=db_con)

In [25]:
# fetch the app_vectors from database

sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        app_vectors = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~app_vectors["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

In [26]:
# json load the embedding in app_vectors
app_vectors['embedding'] = app_vectors['embedding'].apply(json.loads)

# drop the id and content
app_vectors = app_vectors.drop(columns=['id', 'content'])

# Split the 'embedding' column into multiple columns emb_0, emb_1, ..., emb_1023
embedding_split = pd.DataFrame(app_vectors['embedding'].tolist(), columns=[f'emb_{i}' for i in range(len(app_vectors['embedding'].iloc[0]))])

# Concatenate the split columns with the original DataFrame
app_vectors = pd.concat([app_vectors, embedding_split], axis=1)

# drop the original embedding column
app_vectors = app_vectors.drop(columns=['embedding'])

# merge the auctions df with the app_vectors df
df = pd.merge(auctions, app_vectors, on='bundleId', how='left')

In [27]:
# drop unneeded columns
columns_to_drop = ['id', 'deviceId', 'bidFloorPrice']
df.drop(columns=columns_to_drop, inplace=True)

# extract the os from osAndVersion
df['os'] = df['osAndVersion'].str.split('-', expand=True)[0]

# drop the osAndVersion
df.drop(columns=['osAndVersion'], inplace=True)

In [28]:
# one hot encode unneeded columns
catergorial_columns = []
df = pd.get_dummies(df, columns=['unitDisplayType', 'brandName', 'bundleId', 'countryCode', 'os'], dtype=int)

### Fitting and testing the model

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [31]:
# split dataset into train and test
x = df.drop(columns=['sentPrice'])
y = df['sentPrice']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [32]:
regressor = DecisionTreeRegressor()

# fit
regressor.fit(x_train, y_train)

# predict
y_pred = regressor.predict(x_test)

# get the mean squared error
mse = mean_squared_error(y_test, y_pred)

mse

68.3513227400914

### Importance of each feature

In [33]:
"""
function that gets a prefix and a dataframe
it drops all the columns which start with prefix
it trains and tests the model
returns the mse
"""
def get_mse_without_columns(prefix, df):
    columns_to_drop = [col for col in df.columns if col.startswith(prefix)]
    new_df = df.drop(columns=columns_to_drop)
    x = new_df.drop(columns=['sentPrice'])
    y = new_df['sentPrice']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    
    regressor = DecisionTreeRegressor()
    regressor.fit(x_train, y_train)
    y_pred = regressor.predict(x_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

In [34]:
# for each feature x, drop it, train and test the model without it, print the mse

prefixes = ['unitDisplayType', 'brandName', 'bundleId', 'countryCode', 'os', 'emb', 'eventTimestamp']
for prefix in prefixes:
    mse = get_mse_without_columns(prefix, df)
    print(f'mse without {prefix}: {mse}')

mse without unitDisplayType: 86.69606473253326
mse without brandName: 76.98026589688456
mse without bundleId: 68.57012122211619
mse without countryCode: 101.87964196395454
mse without os: 68.40702782004813
mse without emb: 54.921096769188445
mse without eventTimestamp: 31.455016007569064


# Result

### we got the following results:
base mse: 68

mse without unitDisplayType: 86

mse without brandName: 75

mse without bundleId: 68

mse without countryCode: 101

mse without embeddings: 54

mse without eventTimestamp: 31

# Conclusion

we can conclude that the following features are useful and decreases the error of the model (ordered from most important to least important):

1. countryCode
2. unitDisplayType
3. brandName

while the following features are bad and increases the error of the model:

1. embeddings
2. eventTimestamp